In [3]:
from Helper_Utilities import book_researcher_checktool_chain

In [19]:
res = ' tôi đã có thông tin sách mà bạn muốn'

In [20]:
book_researcher_checktool_chain.invoke({'messages': [res]})['messages'] 

'yes'

In [4]:
from Tools import *

In [31]:
load_book("7,8,9")

ID:  {'id': '7,8,9'}
Request succeeded with status 200 (OK)


' Load the book sucessfully '

In [30]:
load_book("5,6,7")

ID:  {'id': '5,6,7'}
Request succeeded with status 200 (OK)


' Load the book sucessfully '

In [5]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from API_keys import *
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


### Construct retriever ###
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [12]:
from langchain_core.messages import HumanMessage,AIMessage

OpenAIHistoryConversation = [HumanMessage(content='thư viện có sách về cơ khí không'),
AIMessage(content='Đã tìm thấy sách cơ khí bạn cần , đó là cuốn Cẩm nang cơ khí có sẵn ở trong thư viện')]

second_question = ' thư viện có cuốn đó không?'
# ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": OpenAIHistoryConversation})

# print(ai_msg_2["answer"])

In [6]:
from langchain.memory import ConversationKGMemory
from langchain.chains import ConversationChain
from langchain_core.prompts.prompt import PromptTemplate

template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)
conversation_with_kg = ConversationChain(
    llm=llm, verbose=True, prompt=prompt, memory=ConversationKGMemory(llm=llm)
)

In [15]:
ai_msg_2 = conversation_with_kg.invoke({"input": second_question, "history": OpenAIHistoryConversation})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human:  thư viện có cuốn đó không?
AI:

> Finished chain.


In [7]:
conversation_with_kg.predict(input="thư viện có sách về cơ khí không")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: thư viện có sách về cơ khí không
AI:

> Finished chain.


'Có, thư viện chứa một bộ sưu tập sách về cơ khí. Có sách về cơ khí cơ bản, cơ khí ứng dụng và cơ khí chuyên sâu. Bạn có thể tìm thấy thông tin chi tiết về cơ khí trong các cuốn sách này. Nếu bạn cần thêm thông tin cụ thể, hãy cho tôi biết!'

In [8]:
conversation_with_kg.predict(input='Đã tìm thấy sách cơ khí bạn cần , đó là cuốn Cẩm nang cơ khí có sẵn ở trong thư viện')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

On thư viện: thư viện có sách về cơ khí.

Conversation:
Human: Đã tìm thấy sách cơ khí bạn cần , đó là cuốn Cẩm nang cơ khí có sẵn ở trong thư viện
AI:

> Finished chain.


'Đúng vậy, thư viện chúng tôi có rất nhiều sách về cơ khí, trong đó có cuốn Cẩm nang cơ khí mà bạn cần. Bạn có thể tìm thấy nó ở kệ sách số 3, hàng số 5. Chúc bạn tìm được thông tin hữu ích từ cuốn sách đó!'

In [12]:
conversation_with_kg.predict(input=' tôi tìm cuốn sách Cẩm nang cơ khí ở kệ nào trong thư viện?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human:  tôi tìm cuốn sách Cẩm nang cơ khí ở kệ nào trong thư viện?
AI:

> Finished chain.


'Xin lỗi, tôi không biết cuốn sách Cẩm nang cơ khí ở kệ nào trong thư viện.Bạn có thể hỏi thủ thư hoặc tìm trên hệ thống máy tính của thư viện để biết chính xác vị trí của cuốn sách đó.'